In [20]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import gmean

In [21]:
def get_quality(df_num):
    df_med_dev = df_num.transform(lambda x: abs(x - x.median())).transform(lambda x: (x*0) + x.median() * 1.4826)
    df_mscore = df_num.transform(lambda x: x - x.median()) / df_med_dev
    df_squash = df_mscore.transform(lambda x: 1 / (1 + np.exp(-x)))
    df_squash['quality'] = gmean(df_squash.iloc[:,:],axis=1)
    df_quality = df_squash['quality']
    return df_quality

In [22]:
df_csv = pd.read_csv('C:/Users/Haniff/Documents/GeneralProgramming/Diversified Projects/d3farmer/fsharp/Runs.csv')
dat_len = len(df_csv) - 100

In [23]:
pick_len = int(100 + dat_len/2)

In [24]:
df = df_csv.tail(pick_len)

In [25]:
# df = df_in[['LevelName', 'LegendariesPH', 'ShardsPH']]

In [26]:
df_num = df[['XPPH','LegendariesPH', 'ShardsPH', 'Keys']]

In [27]:
df_quality = get_quality(df_num)

In [28]:
df.tail(10)

,LevelName,Difficulty,XPPH,LegendariesPH,ShardsPH,Keys
162,T7,11,6900,22.2,1189,1
163,T3,7,2000,19.2,537,2
164,T7,11,5500,15.3,850,2
165,Expert,3,944,0.0,344,1
166,T9,13,7800,23.1,1071,2
167,T9,13,9600,18.7,1268,2
168,T3,7,3300,27.4,862,2
169,T9,13,9000,7.8,1038,2
170,T9,13,10600,48.1,1242,1
171,T9,13,7900,14.3,2904,1


In [29]:
df = df.join(df_quality).sort_values('quality', ascending=False)

In [30]:
df.head(10)

,LevelName,Difficulty,XPPH,LegendariesPH,ShardsPH,Keys,quality
141,T9,13,14800,37.3,1653,2,0.848031
153,T9,13,10200,30.2,1287,2,0.782216
152,T9,13,8900,27.4,1287,2,0.762211
170,T9,13,10600,48.1,1242,1,0.753175
148,T9,13,9800,19.2,1376,2,0.739625
133,T7,11,7300,21.7,1375,2,0.727930
167,T9,13,9600,18.7,1268,2,0.725041
139,T9,13,11100,10.6,4524,2,0.721036
157,T7,11,6200,47.6,953,2,0.716728
110,T8,12,10800,12.4,1550,2,0.709944


In [31]:
dfgb = df.groupby(['LevelName'])

In [32]:
def mad(df):
    const_med = df.median()
    df_med = abs(df - const_med)
    v_mad = df_med.median()
    return v_mad

In [33]:
dfgb = dfgb['quality'].agg(['count','median', mad]).reset_index()

In [34]:
df = df.merge(dfgb)

In [35]:
dfgb['total'] = dfgb['count'].sum()

In [36]:
import math
def get_ucb(row):
    div1 = 3 * math.log(row['total']) / row['count']
    min1 = min(0.25, row['mad'] + div1)
    div2 = math.log(row['total']) / row['count']
    ucb = math.sqrt(div2 * min1)
    return ucb + row['median']

In [37]:
dfgb['Next Action'] = dfgb.apply(get_ucb, axis=1)

In [38]:
dfgb.sort_values('Next Action', ascending=False)

,LevelName,count,median,mad,total,Next Action
14,T9,23,0.664841,0.052266,136,0.895922
4,T11,3,0.254550,0.022800,136,0.894384
12,T7,16,0.613503,0.067242,136,0.890559
3,T10,3,0.249061,0.017311,136,0.888896
1,Master,6,0.421785,0.020303,136,0.874216
5,T12,3,0.231750,0.000000,136,0.871584
6,T13,3,0.231750,0.000000,136,0.871584
0,Expert,4,0.316756,0.008184,136,0.870868
8,T3,7,0.449032,0.012746,136,0.867901
13,T8,22,0.630532,0.038482,136,0.866806
